In [8]:
import statistics
import functools
import string
import random

#### Comparison Function to Key
Map any function which compares elements 
```
    Negative -> a < b
    Zero -> a == b
    Positive -> a > b
```
and converts it into a key which can be passed to sorted, various heap functions etc...

In [35]:
random_str = lambda : "".join(random.choices(string.ascii_lowercase, k=random.randint(1, 5)))

In [41]:
def cmp_function(a, b):
    return len(a) - len(b)

sorting_function = functools.cmp_to_key(cmp_function)
l = [random_str() for _ in range(10)]
print(sorted(l, key=sorting_function))
print(sorted(l, key=len ))

['q', 'ts', 'ad', 'lhg', 'vrv', 'ocz', 'lvrn', 'psrs', 'ceohi', 'vqmiw']
['q', 'ts', 'ad', 'lhg', 'vrv', 'ocz', 'lvrn', 'psrs', 'ceohi', 'vqmiw']


#### LRU Cache
When we are computing the same thing over and over again, LRU Cache can be used to save and recycle an old result

In [69]:
def fibo(n):
    if n == 1 or n == 2:
        return 1
    return fibo(n-1) + fibo(n-2)

@functools.lru_cache(maxsize=200)
def lru_fibo(n):
    if n == 1 or n == 2:
        return 1
    return fibo(n-1) + fibo(n-2)

def dp_fibo(n):
    if n==1 or n==2:
        return 1
    n_1, n_2 = 1, 1
    for _ in range(2, n):
        n = n_1 + n_2
        n_2 = n_1
        n_1 = n
    return n

In [53]:
%%timeit
fibo(25)

21.1 ms ± 1.74 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [54]:
%%timeit
lru_fibo(25)

97.6 ns ± 2.92 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


In [71]:
%%timeit
dp_fibo(25)

1.49 µs ± 3.95 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


### Total Ordering
If you pass eq, and one of (lt, gt, le, ge), the total ordering decorator will infer the rest

In [79]:
@functools.total_ordering
class Student:
    def __init__(self, firstname, lastname, gpa):
        self.firstname = firstname
        self.lastname = lastname
        self.gpa = gpa
        
    def __eq__(self, other):
        return ((self.lastname.lower(), self.firstname.lower()) ==
                (other.lastname.lower(), other.firstname.lower()))
    
    def __lt__(self, other):
        return self.gpa < other.gpa

In [80]:
Mike = Student("Mike", "Berns", 3.2)
David = Student("David", "Race", 4.0)

In [83]:
Mike == Mike, Mike == David, Mike < David, Mike <= David, David > Mike, David >= Mike

(True, False, True, True, True, True)

### Partial
Modify an existing function, keep all the same args

In [124]:
nums = list(range(10))
add = lambda x, y : x + y
add_4 = functools.partial(add, 4)
# note this is eq to add_4 = lambda x: x + 4
print(list(map(add, nums, [4]*10)))
print(list(map(add_4, nums)))

[4, 5, 6, 7, 8, 9, 10, 11, 12, 13]
[4, 5, 6, 7, 8, 9, 10, 11, 12, 13]


### Partial Method
Dont fully appreciate the utility of this one

In [115]:
class Live:
    def __init__(self):
        self._live = False
    def set_live(self,state:'bool'):
        self._live = state
    def __get_live(self):
        return self._live
    def __call__(self):
        # enable this to be called when the object is made callable.
        return self.__get_live()

    # partial methods. Freezes the method `set_live` and `set_dead`
    # with the specific arguments
    set_alive = functools.partialmethod(set_live, True)
    set_dead = functools.partialmethod(set_live, False)

live = Live() # create object
print(live()) # make the object callable. It calls `__call__` under the hood
live.set_alive() # Call the partial method
print(live())

False
True


### Reduce

In [132]:
from operator import add, mul
from math import factorial

In [130]:
triangle_sum = lambda n : n*(n+1)/2
functools.reduce(add, list(range(1,11))), triangle_sum(10)

(55, 55.0)

In [133]:
pi = lambda l : functools.reduce(mul, l)
pi(list(range(1, 10))), factorial(10)

(362880, 3628800)

### Single Dispatch

In [155]:
@functools.singledispatch
def iterate(arg):
    print("parent function called")

def l_t_s_iter(arg: (list, tuple, set), verbose=False):
    print(type(arg))
    for i, val in enumerate(arg):
        print(val, end= " ")
    print()

def d_iter(arg: dict, verbose=False):
    print(type(arg))
    for key in arg.keys():
        print(arg[key], end= " ")
    print()

In [156]:
d = {
    'foo' : 1,
    'bar' : 2
}

l = [1, 2]; s = {1, 2}; t = (1, 2)
iterate.register(list, l_t_s_iter)
iterate.register(set, l_t_s_iter)
iterate.register(tuple, l_t_s_iter)
iterate.register(dict, d_iter)
iterate(d); iterate(l); iterate(s); iterate(t)

<class 'dict'>
1 2 
<class 'list'>
1 2 
<class 'set'>
1 2 
<class 'tuple'>
1 2 
